https://leetcode.com/problems/longest-palindromic-substring/submissions/

5. Longest Palindromic Substring
Medium

Given a string s, return the longest palindromic substring in s.


Example 1:

Input: s = "babad"
Output: "bab"
Note: "aba" is also a valid answer.
Example 2:

Input: s = "cbbd"
Output: "bb"
Example 3:

Input: s = "a"
Output: "a"
Example 4:

Input: s = "ac"
Output: "a"
 

Constraints:

1 <= s.length <= 1000
s consist of only digits and English letters.

In [314]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        s = "#".join(s)
        s = "#" + s + "#"
        n = len(s)
        longest_palindrome = { "length": 0, "coordinate": [0, 0]}
        for i in range(n):
            coordinate = self.find_palindrome(s, i, n)
            if coordinate[1] - coordinate[0] > longest_palindrome["length"]:
                longest_palindrome["length"] = coordinate[1] - coordinate[0]
                longest_palindrome["coordinate"] = coordinate

        s = s[longest_palindrome["coordinate"][0]:longest_palindrome["coordinate"][1]+1]
        w = s.replace("#", "")
        return w
        
    def find_palindrome(self, a, c, n):
        r = min(c, n - c - 1)
        for i in range(r + 1):
            if a[c - i] != a[c + i]:
                return [c-i + 1, c+i - 1]
        return [c - r, c + r]

# Manacher's Algorithm

For the expanding center method.

If the string is "abcd" then the number of iteration is n.

But if the string is "aaaaaaa", the nth charater is visited nth times, and since we iterate through the array, the runtime is O(n^2).

The Manacher's Algorithm uses the symmetric property of palindromes and storing past computation of palindrome radius.

## When the sub palindromes is totally contained in the bigger palindrome

abcd[acdcbwbcdca]kwc

The palindrome `acdcbwbcdca` totally contains 2 palindromes center on w, because palindromes are symmetric.

a[cdc]bwb[cdc]a

Since the symmetric behavour of the contained palindromes is mirrored on w, the radius of the contained palindrome before w is the same as the contained palindrome after w.

If we create an array of palindrome radii.


|index-|0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|
|------|-|-|-|-|-|-|-|-|-|-|--|--|--|--|--|--|--|--|
|string|a|b|c|d|a|c|d|c|b|w| b| c| d| c| a| k| w| c|
|radii |0|0|0|0|0|0|1|0|0|5|

We can copy the mirrored radii from index 8 to 4 to 10 to 14.
Since we know that [cdc] has at most radius 1, because the letters to the left and right of it is not equal, a[cdc]b and that quality is mirrored to the right of w.

|index-|0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|
|------|-|-|-|-|-|-|-|-|-|-|--|--|--|--|--|--|--|--|
|string|a|b|c|d|a|c|d|c|b|w| b| c| d| c| a| k| w| c|
|radii |0|0|0|0|0|0|1|0|0|5| 0| 0| 1| 0| 0|

## When the sub palindromes is not contained in the bigger palindrome

abcd[acdcawacdca]wakwc


|index-|0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|
|------|-|-|-|-|-|-|-|-|-|-|--|--|--|--|--|--|--|--|--|
|string|a|b|c|d|a|c|d|c|a|w| a| c| d| c| a| w| a| w| c|
|radii |0|0|1|0|0|0|2|0|0|5| 0| 0|  |  |  |


We have 2 sub palindromes that extend to the edge of the bigger palindrome.

[acdca]w[acdca]

Since the behavour is mirrored we cannot say that the radius of [acdca] is at most 2 because the values to the left and wright of [acdca] extends beyond the bigger palindrome and cannot say that the vales to the left and right are not equal. 

So to calculate the value of "d" at index 12 we need to expand from index 12 to find the longest palindrome. We have the advantage of mirrored property of the palindrome in which we know that its mirrored value on "w" at index 6 is at least 2 so we only need to calculate the longest palindrome at index 12 from radius 2 onward.

## When the index is not contained in the bigger palindrome
Then we do not have the advantage of the mirrored behaviour of the palindrome and need to calculate the radius at the index by expanding from index i with radius 0.

# Algorithm

1. pad the string with # from the begining to the end of the string and between every character to ensure that we are dealing with an odd string even if the input string is even.
2. have an array that keeps track of the radius of palindromes of the characters of the string that we iterate over.
3. keep track of the center and radius of the palindrome where center + radius is the right boundary which of the rightmost visited index in the string (through iteration or expanding from center).
4. iterate over the string
4a. when the sub palindromes is totally contained in the bigger palindrome
- - copy the mirrored value (around center) from the radii array into index i
4b. when the sub palindromes is not contained in the bigger palindrome
- - calculate the radius at index i by expanding from radius(using the mirrored value from the radii array) as a starting point
4c. when the index is not contained in the bigger palindrome
- - calculate radus at index i by expanding from radius 0.

In [50]:
class Solution {
    public String longestPalindrome(String s) {
        int length = s.length();
        StringBuilder sPrime = new StringBuilder("#");
        for (char c: s.toCharArray()) {
            sPrime.append(c).append("#");
        }
        return manachersAlgorithm(sPrime.toString(), length*2+1).replace("#", "");

    }

    private String manachersAlgorithm(String s, int length) {
        int[] palindromeRadii = new int[length];
        int center = 0;
        int radius = 0;
        int maxRadius = 0;
        int maxIndex = 0;
        int mirror = 0;
        for (int i = 0; i < length; i++) {
            mirror = 2*center - i >= 0 ?  palindromeRadii[center + center - i] : 0;
            if (i + mirror < center + radius) {
                palindromeRadii[i] = mirror;
            } else {
                int newRadius = findRadius(s, length, i, Math.min(center + radius - i, mirror));
                palindromeRadii[i] = newRadius;
                if (i + newRadius >= center + radius) {
                    center = i;
                    radius = newRadius;
                    if (radius > maxRadius) {
                        maxRadius = radius;
                        maxIndex = center;
                    }
                }
            }
        }
        return s.substring(Math.max(0, maxIndex - maxRadius), Math.min(length, maxIndex + maxRadius+1));
    }

    private int findRadius(String s, int length, int center, int radius) {
        while (center - radius - 1 >= 0 && center + radius + 1 < length) {
            if (s.charAt(center + radius + 1 ) != s.charAt(center - radius - 1)) {
                break;
            }
            radius++;
        }
        return radius;
    }
}